In [62]:
!pip install ansi2html

In [61]:
text = """Добрий день я вам був писав в мене були винекли
деякі проблеми якщо можна я завтра буду на робочому місті ?"""

In [63]:
import ast
import os
import re
import nltk
import mlflow
import pandas as pd
import tiktoken
from dotenv import load_dotenv
from omegaconf import OmegaConf
from openai import OpenAI
from flatten_dict import flatten

from src.prompts.reddit_multigec import multi_gec_prompt_per_language
from src.utils.metrics import average_edit_distance
from src.utils.utils import normalize_spaces, generate_original_corrected_texts
from src.utils.comparison_to_html import save_comparison_to_html
from src.prompts.reddit_multigec import gec_aggregation_prompt_per_language

In [64]:
load_dotenv("../../.env")

True

In [65]:
parameters = OmegaConf.load("./parameters.yaml")
mlflow.set_tracking_uri(os.environ.get("MLFLOW_TRACKING_URI"))
mlflow.set_experiment(parameters.experiment.experiment_name)

<Experiment: artifact_location='mlflow-artifacts:/875882461670179036', creation_time=1731081700081, experiment_id='875882461670179036', last_update_time=1731081700081, lifecycle_stage='active', name='reddit_ua_exploratory', tags={}>

In [66]:
tokenizer = tiktoken.encoding_for_model(parameters.tokenizer.model_name)

In [67]:
input_text_tokens = len(tokenizer.encode(text))
input_text_tokens

32

In [68]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [70]:


language = "ukrainian"


grammar_correction_prompt = multi_gec_prompt_per_language[language].prompt_template

grammar_correction_prompt_formatted = grammar_correction_prompt.format(
    text=text,
    num_corrections=parameters.multi_gec.num_corrections
)
print(grammar_correction_prompt_formatted)

Виправте наступний текст, зробивши його граматично правильним.
Виправте всі орфографічні, пунктуаційні, стилістичні, граматичні, лексичні та синтаксичні помилки.
Якщо помилок немає, повторіть оригінальний текст.
Згенеруйте 3 різні варіанти виправленого тексту з поясненнями.

Формат відповіді у JSON:
[{
    "correction": "виправлений текст",
    "explanation": "пояснення до виправлення"
}, ...]

Приклади:

1. Вхідний текст:
   Останні 3 місяці мого життя видалися аж занадто насиченими на події та емоції, але ось нарешті у мене з’явилося декілька вільних годин та трохи енергії щоб продовжити серію записів щодо мого досвіду блогерства.

   Виправлення:
   [("Останні три місяці мого життя були надзвичайно насиченими подіями та емоціями, але нарешті у мене з’явилося кілька вільних годин та трохи енергії, щоб продовжити серію записів щодо свого досвіду блогерства.", "Замінено '3' на 'три' для узгодженості стилю; замінено 'декілька' на 'кілька' для кращого стилю; додано кому після 'енергії' в

In [71]:
grammar_correction_prompt_tokens = len(tokenizer.encode(grammar_correction_prompt.template))
grammar_correction_prompt_formatted_tokens = len(tokenizer.encode(grammar_correction_prompt_formatted))
grammar_correction_prompt_tokens, grammar_correction_prompt_formatted_tokens

(3235, 3260)

In [72]:
len(grammar_correction_prompt_formatted), len(grammar_correction_prompt.template)

(9649, 9566)

In [73]:
client = OpenAI(api_key=os.environ.get("OPEN_AI_API_KEY"))
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": grammar_correction_prompt_formatted,
        }
    ],
    temperature=parameters.multi_gec.temperature,
    top_p=parameters.multi_gec.top_p,
    model=parameters.multi_gec.model_name,
)
original_text = text
multi_gec_raw_output = chat_completion.choices[0].message.content
import json
pattern = r'```json\s*\n(?P<json>([\s\S]*?))\n```'
matches = re.finditer(pattern, multi_gec_raw_output, re.MULTILINE)
for match in matches:
    json_content = match.group('json')
    try:
        multi_gec_output = json.loads(json_content)
        print(multi_gec_output)
    except json.JSONDecodeError as e:
        print("Invalid JSON:", e)
else:
    multi_gec_output = ast.literal_eval(multi_gec_raw_output.strip("```json"))

multi_gec_output_tokens = len(tokenizer.encode(multi_gec_raw_output))
multi_gec_correction_output_tokens = len(tokenizer.encode(str([output["correction"] for output in multi_gec_output])))
multi_gec_explanation_output_tokens = len(tokenizer.encode(str([output["explanation"] for output in multi_gec_output])))

multi_gec_output_tokens, multi_gec_correction_output_tokens, multi_gec_explanation_output_tokens

[{'correction': 'Добрий день! Я вам писав, у мене виникли деякі проблеми. Якщо можна, я завтра буду на робочому місці?', 'explanation': "Додано знак оклику після 'Добрий день' для завершення привітання; виправлено 'винекли' на 'виникли' для правильного написання; додано коми для розділення частин речення; виправлено 'місті' на 'місці' для правильного вживання."}, {'correction': 'Добрий день! Я писав вам, у мене виникли деякі проблеми. Якщо можливо, я завтра буду на робочому місці.', 'explanation': "Змінено порядок слів для покращення стилю; виправлено 'винекли' на 'виникли'; замінено 'якщо можна' на 'якщо можливо' для більш формального звучання; виправлено 'місті' на 'місці'."}, {'correction': 'Добрий день! Я писав вам, у мене виникли деякі проблеми. Чи можу я завтра бути на робочому місці?', 'explanation': "Змінено 'якщо можна' на 'чи можу я' для покращення формулювання питання; виправлено 'винекли' на 'виникли'; додано знаки пунктуації для чіткості."}]


(345, 105, 194)

In [74]:
multi_gec_correction_comparison_text: str = ""

for i, correction_reasoning in enumerate(multi_gec_output):
    correction = correction_reasoning["correction"]
    reasoning = correction_reasoning["explanation"]

    text1 = normalize_spaces(original_text)
    text2 = normalize_spaces(correction)

    original_corrected_text = generate_original_corrected_texts(
        original_text=original_text,
        corrected_text=text2)

    multi_gec_correction_comparison_text += f"""
Correction: {i}

Original Text:
{original_corrected_text[0]}

Corrected Text:
{original_corrected_text[1]}

Reasoning:
{reasoning}
    """

print(multi_gec_correction_comparison_text)


Correction: 0

Original Text:
Добрий день я вам був писав в мене були винекли деякі проблеми якщо можна я завтра буду на робочому місті ?

Corrected Text:
Добрий день! Я вам писав, у мене виникли деякі проблеми. Якщо можна, я завтра буду на робочому місці?

Reasoning:
Додано знак оклику після 'Добрий день' для завершення привітання; виправлено 'винекли' на 'виникли' для правильного написання; додано коми для розділення частин речення; виправлено 'місті' на 'місці' для правильного вживання.
    
Correction: 1

Original Text:
Добрий день я вам був писав в мене були винекли деякі проблеми якщо можна я завтра буду на робочому місті ?

Corrected Text:
Добрий день! Я писав вам, у мене виникли деякі проблеми. Якщо можливо, я завтра буду на робочому місці.

Reasoning:
Змінено порядок слів для покращення стилю; виправлено 'винекли' на 'виникли'; замінено 'якщо можна' на 'якщо можливо' для більш формального звучання; виправлено 'місті' на 'місці'.
    
Correction: 2

Original Text:
Добрий день 

In [75]:


correction_aggregation_prompt = gec_aggregation_prompt_per_language[language].prompt_template

correction_aggregation_prompt_formatted = correction_aggregation_prompt.format(
    text=text,
    possible_corrections=str([output["correction"] for output in multi_gec_output]),
    num_corrections=parameters.multi_gec.num_corrections
)
correction_aggregation_prompt_formatted

'Агрегуйте запроповані граматичні виправлення тексту у фінальний граматично правильний текст.\nЗ оригінального тексту та списку з 3 варіантів його виправлення, об\'єднай всі корисні виправлення з цих варіантів та створи фінальний виправлений синтаксично коректний текст.\nУсунь орфографічні, пунктуаційні, стилістичні, граматичні, лексичні та синтаксичні помилки.\nДобав пояснення до агрегованих виправлень та граматичної корекції тексту.\nСфокосуйся на запропонованих виправленнях, та не старайся виправити оригінальний текст самотужки.\nЯкщо помилок немає, поверни оригінальний текст.\n\nФормат запропонованих виправленнь:\n[{\n    "correction": "виправлений текст",\n    "explanation": "пояснення до виправлення"\n},...]\n\n[Output only JSON]\nФормат відповіді у JSON:\n{\n    "correction": "виправлений текст",\n    "explanation": "пояснення до виправлення"\n}\n\nОригінальний текст: Добрий день я вам був писав в мене були винекли\nдеякі проблеми якщо можна я завтра буду на робочому місті ?\nЗа

In [76]:
gec_aggregation_prompt_tokens = len(tokenizer.encode(correction_aggregation_prompt.template))
gec_aggregation_prompt_formatted_tokens = len(tokenizer.encode(correction_aggregation_prompt_formatted))
gec_aggregation_prompt_tokens, gec_aggregation_prompt_formatted_tokens

(316, 439)

In [77]:
client = OpenAI(api_key=os.environ.get("OPEN_AI_API_KEY"))
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": correction_aggregation_prompt_formatted,
        }
    ],
    temperature=parameters.gec_aggregation.temperature,
    top_p=parameters.gec_aggregation.top_p,
    model=parameters.gec_aggregation.model_name,
)
original_text = text
gec_aggregation_raw_output = chat_completion.choices[0].message.content
import json
pattern = r'```json\s*\n(?P<json>([\s\S]*?))\n```'
matches = re.finditer(pattern, gec_aggregation_raw_output, re.MULTILINE)
for match in matches:
    json_content = match.group('json')
    try:
        gec_aggregation_output = json.loads(json_content)
        print(gec_aggregation_output)
    except json.JSONDecodeError as e:
        print("Invalid JSON:", e)
else:
    gec_aggregation_output = ast.literal_eval(gec_aggregation_raw_output.strip("```json"))

gec_aggregation_output_tokens = len(tokenizer.encode(gec_aggregation_raw_output))
gec_aggregation_correction_output_tokens = len(tokenizer.encode(str(gec_aggregation_output["correction"])))
gec_aggregation_explanation_output_tokens = len(tokenizer.encode(str(gec_aggregation_output["explanation"])))


In [78]:
gec_aggregation_output["correction"]

'Добрий день! Я писав вам, у мене виникли деякі проблеми. Якщо можливо, я завтра буду на робочому місці.'

In [79]:
correction = gec_aggregation_output["correction"]
reasoning = gec_aggregation_output["explanation"]

original_text = text

print("\nSentence #", i)
text1 = normalize_spaces(original_text)
text2 = normalize_spaces(correction)

original_corrected_text = generate_original_corrected_texts(
        original_text=original_text,
        corrected_text=text2)

print("Original Text:")
print(original_corrected_text[0])
print()

print("Corrected Text:")
print(original_corrected_text[1])
print()

print("Reasoning:")
print(reasoning)
print()



Sentence # 2
Original Text:
Добрий день я вам був писав в мене були винекли деякі проблеми якщо можна я завтра буду на робочому місті ?

Corrected Text:
Добрий день! Я писав вам, у мене виникли деякі проблеми. Якщо можливо, я завтра буду на робочому місці.

Reasoning:
Виправлення включає коректне використання розділових знаків, зокрема, додавання знака оклику після 'Добрий день' та ком після 'якщо можливо'. Слово 'винекли' виправлено на 'виникли', а 'робочому місті' на 'робочому місці' для правильного вживання терміна. Формулювання 'якщо можна' замінено на 'якщо можливо' для більш формального звучання.



In [80]:
clip_text_in_run_name_in_chars: int = 30

run_name = f"{parameters.experiment.run_name} {text[:clip_text_in_run_name_in_chars] + '...' if len(text) > clip_text_in_run_name_in_chars else text}"

with mlflow.start_run(run_name=run_name) as run:
    # region Log Outputs and Parameters
    mlflow.log_params(flatten(parameters, reducer="dot"))
    gec_aggregation_html_comparison_file_name = save_comparison_to_html(
        original_text,
        correction,
        reasoning,
        "gec_aggregation.output_comparison.html",
    )
    mlflow.log_artifact(gec_aggregation_html_comparison_file_name)

    mlflow.log_artifact(multi_gec_correction_comparison_file_name)
    # endregion

    # region Log texts and results
    mlflow.log_text(text1, artifact_file="original_text.txt")
    mlflow.log_text(text2, artifact_file="corrected_text.txt")
    mlflow.log_text(reasoning, artifact_file="reasoning.txt")
    # endregion

    # region Log prompts
    mlflow.log_text(
        grammar_correction_prompt_formatted,
        artifact_file="multi_gec.prompt_formatted.txt")
    mlflow.log_text(
        grammar_correction_prompt.template,
        artifact_file="multi_gec.prompt.txt")
    mlflow.log_text(
        correction_aggregation_prompt_formatted,
        artifact_file="gec_aggregation.prompt_formatted.txt"
    )
    mlflow.log_text(
        correction_aggregation_prompt.template,
        artifact_file="gec_aggregation.prompt.txt"
    )
    # endregion

    # region Log prompt tokens
    mlflow.log_metric(
        key="multi_gec.prompt_tokens",
        value=grammar_correction_prompt_tokens
    )
    mlflow.log_metric(
        key="multi_gec.prompt_formatted_tokens",
        value=grammar_correction_prompt_formatted_tokens
    )
    mlflow.log_metric(
        key="gec_aggregation.prompt_tokens",
        value=grammar_correction_prompt_tokens
    )
    mlflow.log_metric(
        key="gec_aggregation.prompt_formatted_tokens",
        value=grammar_correction_prompt_formatted_tokens
    )
    # endregion

    # region Log metrics
    edit_distance = nltk.edit_distance(
        original_text,
        correction
    )
    avg_edit_distance = average_edit_distance(
        original_text,
        correction,
    )
    mlflow.log_metric(
        key="gec_aggregation.edit_distance",
        value=edit_distance,
    )
    mlflow.log_metric(
        key="gec_aggregation.avg_edit_distance",
        value=avg_edit_distance,
    )
    for i, output in enumerate(multi_gec_output):
        multi_gec_correction = output["correction"]

        edit_distance = nltk.edit_distance(
            original_text,
            multi_gec_correction
        )
        avg_edit_distance = average_edit_distance(
            original_text,
            multi_gec_correction,
        )
        mlflow.log_metric(
            key=f"multi_gec_correction.{i}.edit_distance",
            value=edit_distance,
        )
        mlflow.log_metric(
            key=f"multi_gec_correction.{i}.avg_edit_distance",
            value=avg_edit_distance,
        )

    mlflow.log_metric(
        key="multi_gec.input_text_tokens",
        value=input_text_tokens
    )
    mlflow.log_metric(
        key="multi_gec.output_tokens",
        value=multi_gec_output_tokens
    )
    mlflow.log_metric(
        key="multi_gec.correction_output_tokens",
        value=multi_gec_correction_output_tokens
    )
    mlflow.log_metric(
        key="multi_gec.explanation_output_tokens",
        value=multi_gec_explanation_output_tokens
    )
    mlflow.log_metric(
        key="gec_aggregation.output_tokens",
        value=gec_aggregation_output_tokens
    )
    mlflow.log_metric(
        key="gec_aggregation.correction_output_tokens",
        value=gec_aggregation_correction_output_tokens
    )
    mlflow.log_metric(
        key="gec_aggregation.explanation_output_tokens",
        value=gec_aggregation_explanation_output_tokens
    )
    # endregion


HTML file with colored output saved as gec_aggregation.output_comparison.html
HTML file with colored output saved as multi_gec.output_comparison.html


2024/11/11 14:31:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run experiment Добрий день я вам був писав в ... at: http://127.0.0.1:5000/#/experiments/875882461670179036/runs/fd0ffef943cb46e394a9c4aa548e50d2.
2024/11/11 14:31:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/875882461670179036.
